<a href="https://colab.research.google.com/github/lihiShlomit/DSproject/blob/main/Copy_of_Copy_of_trainYolo2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
rm -rf /content/yolov5

In [5]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 17516, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 17516 (delta 7), reused 0 (delta 0), pack-reused 17496 (from 4)
Receiving objects: 100% (17516/17516), 16.66 MiB | 16.64 MiB/s, done.
Resolving deltas: 100% (11998/11998), done.


In [11]:
import os
import sys

print("sys.path:")
for p in sys.path:
    print(p)

print("\nתוכן /content/yolov5:")
print(os.listdir('/content/yolov5'))

print("\nתוכן /content/yolov5/models:")
print(os.listdir('/content/yolov5/models'))

sys.path:
/content/yolov5
/content
/env/python
/usr/lib/python311.zip
/usr/lib/python3.11
/usr/lib/python3.11/lib-dynload

/usr/local/lib/python3.11/dist-packages
/usr/lib/python3/dist-packages
/usr/local/lib/python3.11/dist-packages/IPython/extensions
/root/.ipython
/content/yolov5
/content/yolov5
/content/yolov5

תוכן /content/yolov5:
['README.zh-CN.md', 'data', '.dockerignore', 'pyproject.toml', 'detect.py', 'README.md', 'CITATION.cff', 'hubconf.py', 'models', 'CONTRIBUTING.md', 'utils', 'train.py', 'export.py', 'classify', '.git', 'LICENSE', 'segment', 'tutorial.ipynb', 'val.py', 'benchmarks.py', 'requirements.txt', '.gitignore', '.github', '.gitattributes']

תוכן /content/yolov5/models:
['yolov5n.yaml', 'yolov5m.yaml', 'experimental.py', 'yolov5l.yaml', '__init__.py', 'yolo.py', 'common.py', 'tf.py', 'segment', 'hub', 'yolov5s.yaml', 'yolov5x.yaml']


In [6]:
import os
print(os.path.exists('/content/yolov5/utils/datasets.py'))  # Should now be True

False


In [8]:
!ls /content/yolov5/utils/

activations.py	  callbacks.py	  general.py	     metrics.py
augmentations.py  dataloaders.py  google_app_engine  plots.py
autoanchor.py	  docker	  __init__.py	     segment
autobatch.py	  downloads.py	  loggers	     torch_utils.py
aws		  flask_rest_api  loss.py	     triton.py


In [9]:
import sys
sys.path.append('/content/yolov5')

import torch
from models.yolo import Model
from utils.dataloaders import create_dataloader
from utils.general import check_dataset
from utils.torch_utils import select_device
from utils.loss import ComputeLoss
from pathlib import Path
import os
from utils.general import non_max_suppression
from utils.metrics import bbox_iou
import numpy as np

from torch.utils.data import DataLoader
from utils.dataloaders import LoadImagesAndLabels

# import os
# os.environ['RANK'] = '-1'
# os.environ['WORLD_SIZE'] = '1'
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

ModuleNotFoundError: No module named 'models'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DATA_YAML_PATH = "/content/drive/MyDrive/project/data/license_plates/archive/dataset.yaml"

data = check_dataset(DATA_YAML_PATH)
train_path = data['train']
val_path = data['val']
test_path = data['test']
nc = data['nc']

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def yolo_collate_fn(batch):
    imgs, targets, paths, shapes = zip(*batch)
    imgs = torch.stack(imgs, dim=0)

    # שלב את כל ה-targets לטנזור אחד, והוסף אינדקס תמונה בעמודה 0
    for i, t in enumerate(targets):
        t[:, 0] = i  # קוד YOLO מצפה לעמודה 0 עם אינדקס התמונה
    targets = torch.cat(targets, dim=0)

    return imgs, targets, paths, shapes

In [ ]:

hyp = {
    'mosaic': 1.0,          # הפעלת mosaic augmentation
    'mixup': 0.1,           # הפעלת mixup
    'lr0': 0.001,
    'momentum': 0.9,
    'weight_decay': 0.0005,
    'warmup_epochs': 3.0,
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 0.1,
    'box': 0.05,
    'cls': 0.5,
    'obj': 1.0,
    'anchor_t': 4.0,
    'fl_gamma': 0.0,
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'degrees': 5.0,          # סיבוב עד 5 מעלות
    'translate': 0.1,
    'scale': 0.5,
    'shear': 2.0,            # טיהוי קל
    'perspective': 0.001,    # קצת פרספקטיבה
    'flipud': 0.1,           # הפעלת flip up-down 10%
    'fliplr': 0.5,
    'copy_paste': 0.0,
    'cls_pw': 1.0,
    'obj_pw': 1.0
}

cfg = '/content/yolov5/models/yolov5s.yaml'
nc = 1
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = Model(cfg, ch=3, nc=nc).to(device)

imgsz = 640
batch_size = 16

train_dataset = LoadImagesAndLabels(train_path, imgsz, augment=True, hyp=hyp)
val_dataset = LoadImagesAndLabels(val_path, imgsz, augment=False, hyp=hyp)
test_dataset = LoadImagesAndLabels(test_path, imgsz, augment=False, hyp=hyp)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                          num_workers=4, collate_fn=yolo_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                        num_workers=4, collate_fn=yolo_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                         num_workers=4, collate_fn=yolo_collate_fn)

optimizer = torch.optim.Adam(model.parameters(), lr=hyp['lr0'],
                             betas=(0.9, 0.999), weight_decay=hyp['weight_decay'])

model.hyp = hyp
compute_loss = ComputeLoss(model)

Overriding model.yaml nc=80 with nc=1

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]             

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

epochs = 20
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for imgs, targets, paths, _ in train_loader:
        imgs = imgs.to(device).float() / 255.0
        targets = targets.to(device)
        pred = model(imgs)
        loss, _ = compute_loss(pred, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        torch.cuda.empty_cache()

    avg_loss = total_loss / len(train_loader)
    print(f"\nEpoch {epoch+1}/{epochs} - Train Loss: {avg_loss:.4f}")

    # --- VALIDATION ---
    model.eval()
    val_loss_total = 0
    with torch.no_grad():
        for imgs, targets, paths, _ in val_loader:
            imgs = imgs.to(device).float() / 255.0
            targets = targets.to(device)

            pred = model(imgs)
            loss, _ = compute_loss(pred[1], targets)
            val_loss_total += loss.item()
            torch.cuda.empty_cache()

    avg_val_loss = val_loss_total / len(val_loader)
    print(f"Epoch {epoch+1}/{epochs} - Val Loss: {avg_val_loss:.4f}")

torch.save(model.state_dict(), '/content/drive/MyDrive/project/model_16_20_001.pth')


Epoch 1/20 - Train Loss: 1.7275
Epoch 1/20 - Val Loss: 1.7542

Epoch 2/20 - Train Loss: 1.4055
Epoch 2/20 - Val Loss: 1.1484

Epoch 3/20 - Train Loss: 1.2330
Epoch 3/20 - Val Loss: 0.8804

Epoch 4/20 - Train Loss: 1.1252
Epoch 4/20 - Val Loss: 0.9473

Epoch 5/20 - Train Loss: 1.0569
Epoch 5/20 - Val Loss: 0.9046

Epoch 6/20 - Train Loss: 1.0321
Epoch 6/20 - Val Loss: 0.8706

Epoch 7/20 - Train Loss: 0.9729
Epoch 7/20 - Val Loss: 0.6634

Epoch 8/20 - Train Loss: 0.9436
Epoch 8/20 - Val Loss: 0.6142

Epoch 9/20 - Train Loss: 0.8922
Epoch 9/20 - Val Loss: 0.8056

Epoch 10/20 - Train Loss: 0.8786
Epoch 10/20 - Val Loss: 0.7392

Epoch 11/20 - Train Loss: 0.8714
Epoch 11/20 - Val Loss: 0.8605

Epoch 12/20 - Train Loss: 0.8603
Epoch 12/20 - Val Loss: 0.8251

Epoch 13/20 - Train Loss: 0.8524
Epoch 13/20 - Val Loss: 0.9199

Epoch 14/20 - Train Loss: 0.8282
Epoch 14/20 - Val Loss: 0.6724

Epoch 15/20 - Train Loss: 0.8120
Epoch 15/20 - Val Loss: 0.6970

Epoch 16/20 - Train Loss: 0.8097
Epoch 16/

In [2]:
from yolov5 import YOLOv5
# 1. מגדירים את המודל (או מייבאים אותו)
model = YOLOv5('yolov5s.pt', device=device)

# 2. טוענים משקלים
model.load_state_dict(torch.load('/content/drive/MyDrive/project/model_16_20_001.pth'))

NameError: name 'YOLOv5' is not defined

In [ ]:
import torch
import numpy as np

def validate(model, dataloader, device, iou_thres=0.5, conf_thres=0.001):
    model.eval()
    stats = []  # לאיסוף: (correct, confs, pred_classes, target_classes)

    for imgs, targets, paths, shapes in dataloader:
        imgs = imgs.to(device).float() / 255.0
        targets = targets.to(device)

        with torch.no_grad():
            preds = model(imgs)
            preds = non_max_suppression(preds, conf_thres=conf_thres, iou_thres=0.6)

        for pred in preds:
            if pred is None:
                continue

            correct = torch.zeros(pred.shape[0], dtype=torch.bool, device=device)

            # כל התוויות ב-batch (במקרה שאין אינדקס תמונה)
            labels = targets

            if labels.size(0):
                for ti, t in enumerate(labels):
                    t_box = t[1:5].unsqueeze(0)   # bbox של תווית
                    t_cls = t[0]                 # class id של תווית

                    for pi, p in enumerate(pred):
                        if correct[pi]:
                            continue
                        p_box = p[0:4].unsqueeze(0)  # bbox תחזית
                        p_cls = p[5]                 # class id תחזית

                        iou = bbox_iou(t_box, p_box)
                        if iou > iou_thres and p_cls == t_cls:
                            correct[pi] = True

            stats.append((
                correct.cpu(),
                pred[:, 4].cpu(),  # confidence
                pred[:, 5].cpu(),  # class pred
                labels[:, 0].cpu() # class target
            ))

    # איחוד כל הסטטיסטיקות
    stats = [np.concatenate(x, 0) for x in zip(*stats)] if stats else [np.array([])]*4

    if len(stats) and stats[0].any():
        p, r, ap, f1, _ = ap_per_class(*stats)
        print(f"Precision: {p.mean():.4f}, Recall: {r.mean():.4f}, mAP@0.5: {ap.mean():.4f}")
        return p, r, ap, f1
    else:
        print("No valid predictions for mAP calculation")
        return None


validate(model, val_loader, device)

No valid predictions for mAP calculation


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import joblib
import torch

# שמירה של ה-state_dict של המודל עם joblib
joblib.dump(model.state_dict(), 'yolov5_model.pkl')

In [ ]:
import joblib
import torch
from models.yolo import Model  # ודא שאתה מייבא את המודל בצורה נכונה

# יצירת המודל מחדש עם אותם פרמטרים
cfg = 'models/yolov5s.yaml'  # הקובץ שלך
nc = 1  # מספר המחלקות שלך
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = Model(cfg, ch=3, nc=nc).to(device)

# טעינת ה-state_dict של המודל עם joblib
model.load_state_dict(joblib.load('yolov5_model.pkl'))

# הפיכת המודל למצב של אימות
model.eval()


# חיזוי על נתונים חדשים (כמו תמונה שהכנת מראש)
with torch.no_grad():  # לא צריך לחשב גרדיאנטים בזמן החיזוי
    predictions = model(input_images)  # תחזור כאן על חיזוי על תמונה


# הערכת ביצועי המודל על נתוני בדיקה
validate(model, test_loader, device)

In [ ]:
def validate_debug(model, dataloader, device, iou_thres=0.5, conf_thres=0.25, max_batches=2):
    model.eval()
    printed = 0

    for batch_i, (imgs, targets, paths, shapes) in enumerate(dataloader):
        imgs = imgs.to(device).float() / 255.0
        targets = targets.to(device)

        with torch.no_grad():
            preds = model(imgs)
            preds = non_max_suppression(preds, conf_thres=conf_thres, iou_thres=iou_thres)

        for i, pred in enumerate(preds):
            if pred is None:
                print(f"\nImage {paths[i] if paths else i}: No predictions")
                continue

            print(f"\nImage {paths[i] if paths else i}:")
            print("Ground Truth:")
            gt = targets  # אם אין אינדקס תמונה – כך
            for t in gt:
                print(f"  Class: {int(t[0])}, BBox: {t[1:].tolist()}")

            print("Predictions:")
            for p in pred:
                bbox = p[0:4].tolist()
                conf = float(p[4])
                cls = int(p[5])
                print(f"  Class: {cls}, Conf: {conf:.2f}, BBox: {bbox}")

            printed += 1
            if printed >= max_batches:
                return
validate_debug(model, val_loader, device)


NameError: name 'model' is not defined